# Generación de Embeddings y Evaluación de Similitud de Frases con S-BERT


En este notebook, utilizaremos un modelo S-BERT (Sentence-BERT) pre-entrenado para generar embeddings de frases y evaluar la similitud entre ellas. También evaluaremos el rendimiento del modelo S-BERT utilizando datasets conocidos como STSB (Semantic Textual Similarity Benchmark) y MNLI (Multi-Genre Natural Language Inference).

**Objetivos del notebook:**
1. Cargar un modelo S-BERT pre-entrenado para generar embeddings de frases.
2. Calcular la similitud de coseno entre frases aleatorias y analizar cuáles son más similares.
3. Evaluar el rendimiento del modelo S-BERT utilizando los datasets STSB o MNLI.
    

In [1]:
# Instalamos las librerías necesarias si no están ya instaladas
!pip install sentence-transformers scikit-learn -q 


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Cargar un modelo S-BERT pre-entrenado y generar embeddings de frases

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Cargar un modelo S-BERT pre-entrenado
sbert_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Lista de frases aleatorias para generar embeddings
frases = [
    "El cielo está despejado y azul.",
    "La tecnología avanza rápidamente.",
    "Es un día perfecto para un paseo.",
    "Es un día ideal para un paseo.",
    "El desarrollo de software es una habilidad importante.",
    "Me gusta ver películas en mi tiempo libre."
]

# Generar los embeddings para las frases usando S-BERT
embeddings = sbert_model.encode(frases)

In [ ]:
# Mostrar los embeddings generados
print("Embeddings generados para las frases:")
print(embeddings)  

## Calcular la similitud de coseno entre las frases

In [3]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcular la matriz de similitud de coseno entre las frases
similarity_matrix = cosine_similarity(embeddings)

# Mostrar la matriz de similitud
print("Matriz de Similitud de Coseno entre las frases:")
print(similarity_matrix)

# Encontrar las frases más similares (excluyendo la diagonal)
np.fill_diagonal(similarity_matrix, 0)
most_similar_pairs = np.unravel_index(np.argmax(similarity_matrix), similarity_matrix.shape)
print(f"Las frases más similares son: '{frases[most_similar_pairs[0]]}' y '{frases[most_similar_pairs[1]]}' con una similitud de {similarity_matrix[most_similar_pairs]:.2f}")
    

Matriz de Similitud de Coseno entre las frases:
[[0.99999994 0.43827826 0.52646255 0.53450996 0.44253314 0.48676413]
 [0.43827826 0.9999997  0.4820129  0.50022745 0.40867415 0.4140717 ]
 [0.52646255 0.4820129  1.0000001  0.8669213  0.48401302 0.62186503]
 [0.53450996 0.50022745 0.8669213  0.9999998  0.5409801  0.64951175]
 [0.44253314 0.40867415 0.48401302 0.5409801  1.0000001  0.4044858 ]
 [0.48676413 0.4140717  0.62186503 0.64951175 0.4044858  1.        ]]
Las frases más similares son: 'Es un día perfecto para un paseo.' y 'Es un día ideal para un paseo.' con una similitud de 0.87


## Evaluar el modelo S-BERT utilizando los datasets STSB o MNLI

In [4]:
from sentence_transformers import evaluation, SentenceTransformer
from datasets import load_dataset

# Cargar el dataset STSB para evaluar la similitud textual
stsb_dataset = load_dataset('stsb_multi_mt', 'es', split='dev')

# Preparar los textos y las etiquetas del dataset para evaluación
sentences1 = stsb_dataset['sentence1']
sentences2 = stsb_dataset['sentence2']
labels = stsb_dataset['similarity_score']

# Crear un evaluador de S-BERT para medir el rendimiento del modelo en el dataset STSB
evaluator = evaluation.EmbeddingSimilarityEvaluator(sentences1, sentences2, labels)
score = evaluator(sbert_model)
print(score)
print(f"Rendimiento del modelo S-BERT en el dataset STSB: {score['pearson_cosine']:.4f}")  

{'pearson_cosine': 0.7058473426134255, 'spearman_cosine': 0.7103037997967742, 'pearson_manhattan': 0.7064111012872616, 'spearman_manhattan': 0.7052518876846171, 'pearson_euclidean': 0.7109221058831185, 'spearman_euclidean': 0.7103037997967742, 'pearson_dot': 0.7058473368780483, 'spearman_dot': 0.7103037997967742, 'pearson_max': 0.7109221058831185, 'spearman_max': 0.7103037997967742}
Rendimiento del modelo S-BERT en el dataset STSB: 0.7058


# Ejercicios

### Ejercicio 1

Piensa una pregunta, y un conjunto de respuestas que se puedan clasificar en varios tipos:
- Que contestan bien a la pregunta
- Que contestan muy mal a la pregunta
- Que no tienen nada que ver

Ejecuta S-BERT para obtener los embeddings tanto de la pregunta, como de las respuestas. 
Calcula la "cosine-similarity" entre la pregunta y las respuestas. Escoge la respuesta que tenga menos valor. 
¿Es la respuesta correcta?

In [7]:
from sentence_transformers import SentenceTransformer, util

sbert_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

question = "What is the capital of Spain?"

responses = [
    "The capital of Spain is Madrid.",  # Respuesta correcta
    "The capital of Spain is Berlin.",  # Respuesta incorrecta
    "Bananas are yellow and sweet.",    # No tiene nada que ver
    "Madrid is a beautiful city in Europe."  # Es relevante pero no es una respuesta directa. 
]

question_embedding = sbert_model.encode(question, convert_to_tensor=True)
responses_embeddings = sbert_model.encode(responses, convert_to_tensor=True)

cosine_similarities = util.pytorch_cos_sim(question_embedding, responses_embeddings)

lowest_similarity_idx = cosine_similarities.argmax().item()

lowest_similarity_response = responses[lowest_similarity_idx]

print("Cosine similarities:", cosine_similarities)
print("Response with the highest similarity:", lowest_similarity_response)

c:\Users\Marcos\Programas\Python3.8\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Cosine similarities: tensor([[0.8436, 0.7397, 0.1346, 0.5989]])
Response with the highest similarity: The capital of Spain is Madrid.
